<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#输入文件队列" data-toc-modified-id="输入文件队列-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>输入文件队列</a></span></li><li><span><a href="#生成样例数据" data-toc-modified-id="生成样例数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>生成样例数据</a></span></li><li><span><a href="#读取样例" data-toc-modified-id="读取样例-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>读取样例</a></span></li><li><span><a href="#参考资料" data-toc-modified-id="参考资料-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>参考资料</a></span></li></ul></div>

# 输入文件队列

我们可以将一个 TFRecord 文件中可以存储多个训练样例，但是当训练数据量较大时，可以将数据分成多个 TFRecord 文件来提高处理效率。TensorFlow 提供了`tf.train.match_filenames_once` 函数来获取符合一个正则表达式的所有文件, 得到的文件列表可以通过 `tf.train.string_input_producer` 函数进行有效的管理。

**tf.train.string_input_producer**

`tf.train.string_input_producer` 函数会使用初始化时提供的文件列表创建一个输入队列，输入队列中原始的元素为文件列表中所有的文件。

- 该函数支持随机打乱文件列表中文件的出队顺序。当设置`shuffle=True` 时, 文件在加入队列之前会被打乱, 所以出队的顺序也是随机的. 随机打乱文件顺序以及加入输入队列的过程会跑在一个单独的线程上, 这样不会影响获取文件的速度. 
- 该函数生成的输入队列可以同时被多个文件读取线程操作, 而且输入队列会将队列中的文件均匀分给不同的线程, 不会出现有些文件被处理过多次而有些文件还没有被处理过的情况.
- 当一个输入队列中的所有文件都被处理后, 它会将初始化时提供的文件列表中的文件全部重新载入队列. 参数 `num_epochs`参数来限制加载初始文件列表的最大轮数. 当所有文件都已经被使用了设定的轮数后, 如果继续尝试取出新的文件, 输入队列会报`OutOfRange` 的错误. 

每次**数据读取函数**从文件中读取数据时, **数据读取函数**首先会判断当前是否已有打开的文件可读, 如果没有或者打开的文件已经读完, 这个函数会从输入队列中出队一个文件, 并从这个文件中读取数据.

# 生成样例数据

我们先生成一个样例数据, 然后使用 `tf.train.match_filenames_once`和`tf.train.string_input_producer`来读取产生的样例数据.

In [1]:
import tensorflow as tf

In [3]:
num_shards = 10 # 文件的个数
instances_per_shard = 5 # 每个文件中的样例个数

for i in range(num_shards):
    # 定义文件名
    filename = ('../../../other/test/data.tfrecords-%.5d-of-%.5d' % (i, num_shards))
    
    # 申请writer
    writer = tf.python_io.TFRecordWriter(filename)
    
    # 将数据封装成Example结构并写入TFRecord中
    for j in range(instances_per_shard):
        # 构建特性
        feature_i = tf.train.Feature(int64_list=tf.train.Int64List(value=[i]))
        feature_j = tf.train.Feature(int64_list=tf.train.Int64List(value=[j]))
        
        # 构建特征map
        features_map = {'i': feature_i, 'j':feature_j}
        features = tf.train.Features(feature=features_map)
        
        # 构造example
        example = tf.train.Example(features=features)
        
        # 样例序列化
        serial = example.SerializeToString()
        
        # 把样例写入文件
        writer.write(serial)
        
    # 关闭writer
    writer.close()

# 读取样例

以下代码展示了利用 `tf.train.match_filenames_once`和`tf.train.string_input_producer`函数的使用。

In [4]:
import tensorflow as tf

In [9]:
tf.reset_default_graph()

# 获取文件列表
files = tf.train.match_filenames_once("../../../other/test/data.tfrecords-*")

# 创建文件输入队列
filename_queue = tf.train.string_input_producer(files, shuffle=False)

# 申请一个Reader
reader = tf.TFRecordReader()

# 读取文件中的样例
_, serialized_example = reader.read(filename_queue)

# 把样例解析为特征
feature_i = tf.FixedLenFeature([], tf.int64)
feature_j = tf.FixedLenFeature([], tf.int64)
features_map = {'i': feature_i, 'j':feature_j}
features = tf.parse_single_example(serialized_example, features=features_map)

with tf.Session() as sess:
    # 虽然在本段程序找那个没有申明任何变量,
    # 但是tf.train.match_filenames_once函数时需要初始化一些变量
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    
    # 打印文件列表
    print("files: ", files)
    
    # 申明tf.train.Coordinator类来协助不同线程，并启动线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 多次执行线程获取数据的操作
    for i in range(6):
        print(sess.run([features['i'], features['j']]))
              
    # 终止线程
    coord.request_stop()
    coord.join(threads)

files:  <tf.Variable 'matching_filenames:0' shape=<unknown> dtype=string_ref>
[0, 0]
[0, 1]
[0, 2]
[0, 3]
[0, 4]
[1, 0]


# 参考资料

1. 《TensorFlow实战google深度学习框架》